##**Diffusion Journal Club: EPI Distortions**

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image
import matplotlib.image as mpimg
from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy import signal
import pywt as wt
from skimage.data import shepp_logan_phantom

font = {'weight' : 'normal',
        'size'   : 18}

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)


## Geometric Distortions:


In [2]:
# Simulate Shepp Logan Phantom and separate fat and water
data_dir = "/content/"
mat_fname = pjoin(data_dir, 'shepplogan.mat')
mat_contents = sio.loadmat(mat_fname)
mat_contents
f = mat_contents['shepp']

# Fat mask
f_fat = np.zeros([256,256])
f_fat[f>0.8] = f[f>0.8]

# Water mask
f_water = np.zeros([256,256])
f_water[f<=0.8] = f[f<=0.8]

sx = len(f)
sy = len(f[:])
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,8))
ax1.imshow(f_fat)
ax2.imshow(f_water)

FileNotFoundError: ignored

**Generate a B0 Field Map With Off-Resonance Effects**

In [ ]:
x, y = np.meshgrid(np.linspace(-1,1,256), np.linspace(-1,1,256))
d = np.sqrt(x*x+y*y)
sigma, mu = 1.0, 0.0
a = 100              # Scale to increase/decrease off-resonance effects
#a = 1000
g = a*np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )

print("2D Gaussian-like array:")
plt.figure(figsize=(17,7))
plt.rc('font', **font)
plt.subplot(1,2,1)
plt.imshow(g,vmin=0,vmax=256)
plt.title('B0 map')

**EPI Signal Model**

$s[m,n] = \int_x \int_y \rho(x,y) e^{i 2 \pi u_m x}  e^{i 2 \pi v_n y}  e^{i 2 \pi g(x,y) \Delta T n }   dx dy $

In [ ]:
# Calculate EPI signal, acquire kspace, and convert back to image space

nx = 256            # Readout steps
ny = 256            # Phase-encoding steps
dT = 500e-6         # Echo Spacing

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
  curf = f*np.exp(1j*2*np.pi*g*dT*(n));
  curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));
  ksp[:,n] = curhatf[:,n];

f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
plt.imshow(np.abs(np.concatenate((f,f2),axis=1)))

In [ ]:
# Same as above, but with fat signal only

nx = 256
ny = 256
dT = 500e-6

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
  curf = f_fat*np.exp(1j*2*np.pi*g*dT*(n));
  curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));
  ksp[:,n] = curhatf[:,n];

f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
plt.imshow(np.abs(np.concatenate((f_fat,f2),axis=1)))

In [ ]:
# Same as above, water signal only

nx = 256
ny = 256
dT = 500e-6

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
    curf = f_water*np.exp(1j*2*np.pi*g*dT*(n));
    curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));
    ksp[:,n] = curhatf[:,n];

f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
plt.imshow(np.abs(np.concatenate((f_water,f2),axis=1)))
plt.show()

**Change Phase Encoding Direction to A/P**

In [ ]:
nx = 256            # Readout steps
ny = 256            # Phase-encoding steps
dT = 500e-6         # Echo Spacing

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
  curf = f*np.exp(1j*2*np.pi*g*dT*(n));
  curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));
  ksp[n,:] = curhatf[n,:];

f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
plt.imshow(np.abs(np.concatenate((f ,f2),axis=1)))

 **Increase/Decrease Echo Spacing**


In [ ]:
nx = 256            # Readout steps
ny = 256            # Phase-encoding steps
dT = 250e-6         # Echo Spacing

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
  curf = f*np.exp(1j*2*np.pi*g*dT*(n));
  curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));
  ksp[:,n] = curhatf[:,n];

f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
plt.imshow(np.abs(np.concatenate((f ,f2),axis=1)))

**Undersample k-space**

In [ ]:
nx = 256            # Readout steps
ny = 256            # Phase-encoding steps
dT = 250e-6         # Echo Spacing

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
  if np.mod(n,2)==1:
    curf = f*np.exp(1j*2*np.pi*g*dT*(n));
    curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));
    ksp[:,n] = curhatf[:,n];
 
f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
plt.imshow(np.abs(np.concatenate((f ,f2),axis=1)))

## Chemical Shift Artifacts

In [ ]:
nx = 256
ny = 256
dT = 500e-6
ppm = 3.3e-6
gamma = 42.57e6
B0 = 3
freq_offset = ppm*gamma*B0    #Chemical Shift @3T

ksp = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
kspw = np.zeros([nx,ny]) + 1j*np.zeros([nx,ny])
for n in range(0,ny):
  curf = f_fat*np.exp(1j*2*np.pi*freq_offset*dT*(n));
  curhatf = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf)));  # Fat signal with shift
  ksp[:,n] = curhatf[:,n];
  curfw = f_water*np.exp(1j*2*np.pi*dT*(n));
  curhatfw = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(curf))); # Water signal
  kspw[:,n] = curhatfw[:,n];

f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(ksp)));
f2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(kspw)));

fig, ax = plt.subplots()
ax.imshow(f_water)
ax.imshow(np.abs(f2), alpha = 0.4)